## Xvectorのデータ準備

In [1]:
from espnet2.bin.tts_inference import Text2Speech
from espnet2.train.dataset import ESPnetDataset
import os

exp_dir = "./exp/tts_finetune_xvector_vits_raw_phn_jaconv_pyopenjtalk_prosody"
train_config_path = os.path.join(exp_dir, "config.yaml")
model_path = os.path.join(exp_dir, "train.total_count.ave.pth")
vocoder_tag = "parallel_wavegan/jsut_parallel_wavegan.v1"
text2speech = Text2Speech.from_pretrained(
    train_config=train_config_path,
    model_file=model_path,
    vocoder_tag=vocoder_tag,
)
dataset = ESPnetDataset([
    ("./dump/22k/xvector/eval1/spk_xvector.scp", "xvector", "kaldi_ark")])

/mnt/data/users/snegishi/M2/espnet/tools/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/mnt/data/users/snegishi/M2/espnet/espnet2/gan_tts/vits/monotonic_align/__init__.py:19: UserWarning: Cython version is not available. Fallback to 'EXPERIMETAL' numba version. If you want to use the cython version, please build it as follows: `cd espnet2/gan_tts/vits/monotonic_align; python setup.py build_ext --inplace`
  warnings.warn(


In [2]:
from IPython.display import display, Audio, Markdown

# i番目のデータを取得する
#   評価セットは1人あたり5データずつあるので，5つごとに個人性が変わる
i = 0
id      = dataset[i][0]        # 発話IDから個人IDを取り出す
xvector = dataset[i][1]['xvector']  # X-Vector（個人性を表すベクトル）

# 合成（spembsにxvectorを指定することで，個人性を表現する）
result = text2speech("それではこれで、明日の状況の確認をお願いしようと思います", spembs=xvector)

# 合成音の表示
display(Markdown(f"**{id}**"))

display(Audio(result["wav"].cpu(), rate=text2speech.fs))


**jvs001**

---

In [3]:
import pandas as pd
import numpy as np

In [4]:
def read_csv(path):
    return pd.read_csv(path,header=None,names=["id","NL"])

In [5]:
def read_mixcsv(path):
    return pd.read_csv(path,header=None,names=["id1","id2","ratio1","ratio2","NL"])

In [6]:
def make_id(id):
    if id < 6:
        id -= 1
    elif id < 28:
        id -= 2
    else:
        id -= 3
    return int(id)

In [7]:
def id2vector(df,dataset):
    output = []
    for i in range(len(df)):
        output_in = []
        id = make_id(df['id'][i])
        NL = df['NL'][i]
        name = dataset[id][0][:6]
        vector = dataset[id][1]['xvector']
        print(f"name :{name}")
        print(f"NL :{NL}")
        # print(f"vector :{len(vector)}")
        output_in.append(NL)
        output_in[1:1] = vector
        # output_in.append(vector)
        output.append(output_in)
    return output

In [8]:
def mixid2vector(df,dataset):
    output = []
    for i in range(len(df)):
        output_in = []
        id1 = make_id(df['id1'][i])
        id2 = make_id(df['id2'][i])
        NL = df['NL'][i]
        vector1 = dataset[id1][1]['xvector']
        vector2 = dataset[id2][1]['xvector']
        mix_vector = vector1*(df['ratio1'][i]/10) + vector2*(df['ratio2'][i]/10)
        output_in.append(NL)
        output_in[1:1] = mix_vector
        output.append(output_in)
    return output

In [9]:
id_and_NL = read_csv('./Xvector_Bert/data/id_and_NL.csv')
mixid_and_NL = read_mixcsv('./Xvector_Bert/data/mix_spk_NL.csv')

In [10]:
len(dataset[10][1]["xvector"])

512

In [11]:
mixid_and_NL

,id1,id2,ratio1,ratio2,NL
0,94,65,5,5,落ち着いた感じの20代ぐらいの女性
1,41,80,2,8,明るく爽やかな感じの男の人
2,67,34,3,7,淡々と話す40代社会人男性
3,55,89,5,5,30代後半ぐらいのおとなしい女の人
4,72,58,5,5,元気な少し声の高い若手OL
...,...,...,...,...,...
96,33,30,2,8,こもった声で早口な女子大学生
97,51,34,5,5,爽やかで元気な男の子
98,42,75,5,5,ゆっくり話すやさしい男性
99,90,14,5,5,高いキュートな声の少女


In [12]:
output_list = id2vector(id_and_NL,dataset)
output_df = pd.DataFrame(output_list,index=None,columns=None)

name :jvs010
NL :おとなしく幼さを感じる女子中学生
name :jvs011
NL :中性的な声の青年
name :jvs014
NL :少年ぽさを感じる女の子
name :jvs015
NL :20歳程度のお姉さん
name :jvs018
NL :ゆっくり間を開けて話す女性
name :jvs020
NL :早口な男性
name :jvs021
NL :ゆっくり淡々と話す40代のおじさん
name :jvs030
NL :ゆっくりと活舌悪くこもり声の女性
name :jvs031
NL :声が高く元気な男子
name :jvs033
NL :間をおいて話す男性
name :jvs034
NL :ゆっくり落ち着きのある低い声の５０代男性
name :jvs035
NL :まったりと話す優しそうな40代女性
name :jvs036
NL :間をおいて話す明るい女子中学生
name :jvs041
NL :ゆったりとした優しそうな男子
name :jvs042
NL :落ち着いた声で話す少し声の低い男性
name :jvs043
NL :少し声の低いおとなしい女子
name :jvs047
NL :70代近いおじいちゃん
name :jvs051
NL :明るくはきはきとした女子
name :jvs052
NL :さわやか系男子高校生
name :jvs054
NL :イントネーションが独特な陽気な男性
name :jvs055
NL :おしとやかな女の子
name :jvs058
NL :おっとりとした声の高いおばあちゃん
name :jvs064
NL :ゆったり話すクールな女子
name :jvs065
NL :40～50歳ぐらいの感情豊かなおばちゃん
name :jvs067
NL :明るくテンションの高い女子高生
name :jvs070
NL :少し早口な感情豊かな30代ぐらいのお兄さん
name :jvs071
NL :マイペースで渋い声なおじさん
name :jvs072
NL :ゆっくり無感情な感じの大人な女性
name :jvs075
NL :テンション高そうな男の人
name :jvs078
NL :クールで少し落ち着いたお兄さん
name :jvs080
NL :人気のありそうな若い陽キャ男子
name :jvs081
NL 

In [13]:
mix_output_list = mixid2vector(mixid_and_NL,dataset)
mix_output_df = pd.DataFrame(mix_output_list,index=None,columns=None)

In [15]:
output_df.to_csv('/mnt/data/users/snegishi/M2/espnet/egs2/jvs_ms_negishi/tts1/Xvector_Bert/data/NL_Xvector.csv',header=False,index=False)

In [16]:
# mix_output_df.to_csv('/mnt/data/users/snegishi/M2/espnet/egs2/jvs_ms_negishi/tts1/Xvector_Bert/data/Mix_NL_Xvector.csv',header=False,index=False)